In [30]:
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import pandas as pd
from tqdm.notebook import tqdm
import multiprocessing as mp
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")   

In [5]:
valid_train_df = pd.read_csv('valid_train_df.csv')
# Add index to DataFrame
print(valid_train_df.head())

         filename  label  split        original  \
0  aagfhgtpmv.mp4      1  train  vudstovrck.mp4   
1  aapnvogymq.mp4      1  train  jdubbvfswz.mp4   
2  abarnvbtwb.mp4      0  train             NaN   
3  abqwwspghj.mp4      1  train  qzimuostzz.mp4   
4  acifjvzvpm.mp4      1  train  kbvibjhfzo.mp4   

                                                path  
0  D:\W\VS\VS Folder\DFD\DFDC MTCNN Extracted\aag...  
1  D:\W\VS\VS Folder\DFD\DFDC MTCNN Extracted\aap...  
2  D:\W\VS\VS Folder\DFD\DFDC MTCNN Extracted\aba...  
3  D:\W\VS\VS Folder\DFD\DFDC MTCNN Extracted\abq...  
4  D:\W\VS\VS Folder\DFD\DFDC MTCNN Extracted\aci...  


In [ ]:
# # Analysis of values

# label_counts = valid_train_df['label'].value_counts()
# label_0_count = label_counts.get(0, 0)
# label_1_count = label_counts.get(1, 0)
# print(f"Label 0 count: {label_0_count}, Label 1 count: {label_1_count}")

Label 0 count: 76, Label 1 count: 306


In [48]:
row=valid_train_df.iloc[192]
print(row['path'],',', row['label'])
path= row['path']
label = row['label']

D:\W\VS\VS Folder\DFD\DFDC MTCNN Extracted\cizlkenljw , 0


In [34]:
model_path = 'vit_backbone.pth'
classifier_path = 'vit_classifier_head.pth'

In [35]:
# Load the model and processor

processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTModel.from_pretrained('google/vit-base-patch16-224')
model = model.to(device)
classifier = nn.Linear(768, 1)
classifier = classifier.to(device)


Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
# Load weights

model.load_state_dict(torch.load(model_path))
classifier.load_state_dict(torch.load(classifier_path))
model = model.to(device)
classifier = classifier.to(device)
model.eval()
classifier.eval()


Linear(in_features=768, out_features=1, bias=True)

In [49]:
batch_size = 36
pred_classes = []
total_batches = 0

image_files = sorted([ os.path.join(path, f)
for f in os.listdir(path)
if f.lower().endswith('.png')
])

for i in tqdm(range(0, len(image_files), batch_size), desc='Image Batches', leave=False):
    total_batches += 1
    batch_paths = image_files[i:i+batch_size]
    images = [Image.open(p).convert("RGB") for p in batch_paths]
    inputs = processor(images=images, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        cls_embeddings = outputs.last_hidden_state[:, 0, :]  # (batch, hidden_dim)
        outputs = classifier(cls_embeddings)  # (batch, 1)
        preds = (torch.sigmoid(outputs) > 0.5).int()
        pred_classes.extend(preds.cpu().numpy().flatten().tolist())

pred = int(sum(pred_classes) > len(pred_classes) / 2)
display(f"Predicted class: {pred}, Actual class: {label}")



Image Batches:   0%|          | 0/9 [00:00<?, ?it/s]

'Predicted class: 1, Actual class: 0'